## Проект. Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем файл с данными и изучим общую информацию. 

Импортируем библиотеку Pandas как pd для удобства. Прочитаем файл data.csv, сохраним его в переменной df и выведем 10 первых строк таблицы для ознакомления.

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Первое впечатление о таблице:
- days_employed: дни с минусом и долями!
- education: есть значения с заглавной буквы/все заглавные
- gender: пол указать в более привычной форме(муж/жен)
- total_income привести в целое число(улучшит восприятие данных)


Получим общую информацию о данных таблицы df, вызвав метод info()

In [3]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Описание данных:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

### Вывод

В значения столбцов days_employed и total_income есть пропуски, в них же значения с плавающей запятой.  
Названия столбцов информативны, нет нужды их форматировать. Оставим названия столбцов как есть.

## Шаг 2. Предобработка данных

### Обработка пропусков

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.  
Воспользуемся методом isnull() и подсчитаем количество для каждого столбца, применив sum()

In [4]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
df.sort_values(by='total_income').tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


Заменим отрицательные значения столбца days_employed на абсолютное методом abs():

In [6]:
df['days_employed'] = df['days_employed'].abs()

Применив метод isnull(), обнаружили пропуски в столбцах days_employed и total_income. Общий трудовой стаж в днях не очень важен в данном иследовании, а вот доход напрямую влияет на результаты. Найдем среднее значение mean() и медиану median() для них. 

In [7]:
days_employed_avg = df['days_employed'].mean()
days_employed_avg

66914.72890682236

In [8]:
days_employed_median = df['days_employed'].median()
days_employed_median

2194.220566878695

Получили 2 значения среднее days_employed_avg и медиану days_employed_median. По среднему, если работать без выходных это 183 года, что не реально. Значения в данном столбце не корректны. Возьмем медиану и заполним пропуски ей для данного столбца.

In [9]:
df['days_employed'] = df['days_employed'].fillna(days_employed_median)

Теперь у нас пропуски только в столбце total_income:

In [10]:
df.isnull().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Подсчитаем количество каждой категории в income_type методом count() и выведем 5 последних строк методом tail()

In [11]:
df.groupby('income_type')['total_income'].count()

income_type
безработный            2
в декрете              1
госслужащий         1312
компаньон           4577
пенсионер           3443
предприниматель        1
сотрудник          10014
студент                1
Name: total_income, dtype: int64

In [12]:
df.sort_values(by='total_income').tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21489,2,2194.220567,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,2194.220567,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,2194.220567,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,2194.220567,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,2194.220567,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


Запишем в переменную jobs все уникальные значения столбца income_type. Далее циклом прогоним все значения jobs, найдем медиану и среднее для каждой категории.

In [13]:
jobs = df['income_type'].unique()

In [14]:
for item in jobs:
    median = df[(df['income_type'] == item)]['total_income'].median()
    mean = df[(df['income_type'] == item)]['total_income'].mean()
    maximum = df[(df['income_type'] == item)]['total_income'].max()
    minimum = df[(df['income_type'] == item)]['total_income'].min()
    print('|{: <16}|{: <10}{: <18}|{: <10}{: <18}|{: <6}{: <18}|{: <6}{: <19}|'.format(item,'медиана =',median, 'среднее =', mean,'max =', maximum, 'min =', minimum))

|сотрудник       |медиана = 142594.39684740017|среднее = 161380.26048788553|max = 1726276.0143316737|min = 21367.648356486974 |
|пенсионер       |медиана = 118514.48641164352|среднее = 137127.4656901654 |max = 735103.2701666558 |min = 20667.26379327158  |
|компаньон       |медиана = 172357.95096577113|среднее = 202417.4614617771 |max = 2265604.028722744 |min = 28702.812888839853 |
|госслужащий     |медиана = 150447.9352830068 |среднее = 170898.30992266268|max = 910451.4708141336 |min = 29200.077192982837 |
|безработный     |медиана = 131339.7516762103 |среднее = 131339.7516762103 |max = 202722.51136840886|min = 59956.9919840117   |
|предприниматель |медиана = 499163.1449470857 |среднее = 499163.1449470857 |max = 499163.1449470857 |min = 499163.1449470857  |
|студент         |медиана = 98201.62531401133 |среднее = 98201.62531401133 |max = 98201.62531401133 |min = 98201.62531401133  |
|в декрете       |медиана = 53829.13072905995 |среднее = 53829.13072905995 |max = 53829.13072905995 |min

Перезапишем пустые значение для каждой категории на медиану, т.к. есть уникальные большие и низкие значения по доходу для категорий, а среднее может исказить данные.  
Проверим результаты, вызвав isnull() и info()

In [17]:
for item in jobs:
    category =  df[(df['income_type'] == item)]
    median = category['total_income'].median()
    df[(df['income_type'] == item)] = df[(df['income_type'] == item)].fillna(median)

In [18]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Пропуски в days_employed могли произойти из за того, что клиент банка не указал трудовой стаж, либо при выгрузке данных, так как стаж указывают в годах в лучшем случае.  
Пропуски в total_income имеют гораздо большее значение, так как на основе дохода рассчитывается кредит. 10% данных было утерено. В этом случае нужно обратиться в кредитный отдел и получить недостающие данные (так же смущает одинаковое количество пропусков в этих двух столбцах, вероятно, это произошло при выгрузке данных)

### Замена типа данных

Заменим значения типа float64 на целочисленные значения int64 в столбцах days_employed и total_income с помощью astype(), проверим результаты, вызвав info().

In [367]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Заменили тип данных в двух столбцах на целочисленные. Это поможет лучше воспринимать числа без долей (с учетом их порядка) и проводить последующие математические операции. 

### Поиск аномальных значений и обработка данных

Найдем и обработаем все анамальных значения и приведем таблицу к удобному для исследованию виду. Далее удалим все дубликаты

In [368]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Найдем все уникальные значения столбцов для поиска аномалий, кроме days_employed и total_income, используя value_counts()

In [369]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значения 20 и -1, вероятно, ввели по ошибке. Так как в сумме оба меньше 1% данных и не повлияют сильно результаты, заменим на 2 и 1 соответственно:

In [370]:
df['children'] = df['children'].replace(20,2).abs()
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [371]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце dob_years нашли 1 аномальное значение 0. Вероятно, не внесли возраст при заполнение сведений о заемщике. Заменим на медиану, и проверим выведя 5 строк отсортированных по dob_years по возрастанию.

In [372]:
years_median = df['dob_years'].median()
years_median

42.0

In [373]:
df['dob_years'] = df['dob_years'].replace(0,42)
df.sort_values(by = 'dob_years', ascending = True).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
766,0,796,19,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,80812,покупка жилья для сдачи
8316,0,556,19,среднее,1,Не женат / не замужем,4,F,сотрудник,0,68524,высшее образование
4098,0,111,19,среднее,1,гражданский брак,1,M,компаньон,0,91876,на проведение свадьбы
9218,0,322,19,среднее,1,гражданский брак,1,F,компаньон,0,103676,сыграть свадьбу
10235,0,793,19,среднее,1,женат / замужем,0,F,сотрудник,0,131308,ремонт жилью


In [374]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

В education образование записано с учетом регистра и наплодило категории образования. Применим метод str.lower() в Pandas и проверим результаты.

In [375]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [376]:
df['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [377]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [378]:
df['family_status_id'].value_counts()


0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

В education_id, family_status и family_status_id все в порядке.

In [379]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В столбце gender аномальное значение XNA. С вероятностью примерно в 66% это женский пол, так количество женщин по данным в 2 раза больше мужчин. Заменим F и M на жен и муж 

In [380]:
df.loc[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


In [381]:
df['gender'] = df['gender'].replace('F', 'жен')
df['gender'] = df['gender'].replace('M', 'муж')
df['gender'] = df['gender'].replace('XNA', 'жен')
df['gender'].value_counts()

жен    14237
муж     7288
Name: gender, dtype: int64

In [382]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [383]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

В столбцах income_type и debt аномалий нет.

In [384]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

В столбце purpose очень много значений, которые можно категоризировать. Категоризируем после лемматизации.

### Обработка дубликатов

Найдем и обработаем дубликаты используя df.duplicated() и посчитаем количество методом sum().  
(Метод value_counts() не подойдет, нет столбцов с уникальными значениями)

In [385]:
df.duplicated().sum()

72

Количество дубликатов меньше 1% данных, на результаты иследования это не повлияет. Применим метод drop_duplicates(), сбросим индексацию строк reset_index() и удалим старую (drop= True). Проверим результат, вызвав df.duplicated().sum()

In [386]:
df = df.drop_duplicates().reset_index(drop= True)

In [387]:
df.duplicated().sum()

0

### Вывод

В данных присутствуют 72 дубликатов целых строк, количество меньше 1% и не повлияет на исследования, поэтому их удалили. Вероятно, они образовались при выгрузке данных. 
Так же есть аномальные значения, возникшие вероятно из за человеческого фактора и ввода информации о клиенте самим оператором. Все аномалии заменены на подходящие для анализа.

### Лемматизация

Лемматизируем столбец purpose, с целью дальнейшего категоризирования. Используем библиотеку pymystem3.  
Для того, чтобы лемматизировать каждое значение в столбце purpose, напишем функцию purpose_group, которая принимает значение столбца purpose и возвращает обработанный результат.  
Создадим новый столбец purpose_group со значениями после обработки

In [388]:
from pymystem3 import Mystem
m = Mystem()

def purpose_group (purpose):
    lemmas = m.lemmatize(purpose)
    return lemmas

df['purpose_group'] = df['purpose'].apply(purpose_group)
df['purpose_group'].value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       767
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

### Вывод

Леммматизировали столбец purpose, используя библиотеку pymystem3, т.к. она для русского языка и выводит начальную форму слова. Это необходимо для поиска совпадений слов в значениях столбца purpose.

### Категоризация данных

Напишем еще одну функцию purpose_update, которая принимает значение столбца purpose_group и категоризирует согласно условию.  
Итого после категоризации получем 4 категории: автомобиль, свадьба, образование и недвижимость.  
Создадим новый столбец purpose_update из обработанных данных. Выведем 5 верхних строк и проверим на уникальность.

In [389]:
def purpose_update(purpose):
    if 'автомобиль' in purpose:
        return 'автомобиль'
    elif 'свадьба' in purpose:
        return 'свадьба'
    elif 'образование' in purpose:
        return 'образование'
    else:
        return 'недвижимость'
    
df['purpose_update'] = df['purpose_group'].apply(purpose_update)
df.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,purpose_update
0,1,8437,42,высшее,0,женат / замужем,0,жен,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,жен,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,муж,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,муж,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266,53,среднее,1,гражданский брак,1,жен,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба


In [390]:
df['purpose_update'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2323
Name: purpose_update, dtype: int64

Для категоризации по доходу воспользуемся квартилями. Медианное значение по столбцу total_income присвоим переменной median_income, высчитаем нижний q1(25%) и верхний квартили q3(75%) соответственно.

In [391]:
median_income = df['total_income'].median()
q1 = df[(df['total_income'] <= median_income)]['total_income'].median()
q3 = df[(df['total_income'] > median_income)]['total_income'].median()
print(q1)
print(median_income)
print(q3)

108075.0
142594.0
196593.0


Напишем функцию income_group, которая принимает значение столбца income_group и категоризирует доход согласно условию.  
Создадим новый столбец income_group из обработанных данных.

In [392]:
def income_group(income):
    if income < q1 :
        return 'Меньше {} руб.'.format(q1)
    elif q1 <= income < median_income:
        return 'От {} до {} руб.'.format(q1, median_income)
    elif median_income <= income < q3:
        return 'От {} до {} руб.'.format(median_income, q3)
    elif q3 <= income:
        return 'От {} руб.'.format(q3)
    
df['income_group'] = df['total_income'].apply(income_group)
df['income_group'].value_counts()

От 142594.0 до 196593.0 руб.    6374
Меньше 108075.0 руб.            5421
От 196593.0 руб.                5306
От 108075.0 до 142594.0 руб.    4352
Name: income_group, dtype: int64

### Вывод

В категоризации по целям кретита жилье и недвижемость отнесем к одной категории, т.к. относиться к недвижемому имуществу и операции производимые с ними схожи.  
В категоризации по доходу использовали медианное значение и нижний и верхний квартили, т.к. в доходах присутствуют уникально большие и низкие значения, а медианные значения позволят получить более реальные данные. 

## Шаг 3. Ответим на вопросы

### - Есть ли зависимость между наличием детей и возвратом кредита в срок?

Создадим вспомогательный столбец credits, для того чтобы сформировать сводную таблицу (это наличие кредита - 1)
Сформируем сводную таблицу методом pivot_table() по категории количества детей children и столбцам задолжности debt, числа кредитов credits, и отношения debt к credits. С помощью метода merge() присоеденим все столбцы в одну таблицу.
Изменим названия столбцов методом set_axis() и отсортируем процент по убыванию.
Далее данный код будет использоваться для каждой категории, изменяя лишь столбец исследования.

In [393]:
df['credits'] = df['debt'].count() / df['debt'].count()
data_pivot_debt = df.pivot_table(index=['children'], columns='credits', values='debt', aggfunc='sum')
data_pivot_credits = df.pivot_table(index=['children'], columns='credits', values='debt', aggfunc='count')
data_ratio = data_pivot_debt / data_pivot_credits * 100
result = data_pivot_debt.merge(data_pivot_credits, on='children')
result = result.merge(data_ratio, on='children')
result.set_axis(['Кредиты с задолжностью', 'Количество кредитов', 'Процент задолжности, %'], axis= 'columns', inplace= True)
result.sort_values(by='Процент задолжности, %', ascending= False)

,Кредиты с задолжностью,Количество кредитов,"Процент задолжности, %"
children,,,
4,4,41,9.756098
2,202,2128,9.492481
1,445,4855,9.165808
3,27,330,8.181818
0,1063,14090,7.544358
5,0,9,0.000000


### Вывод

Согласно полученным данным, заемщики без детей почти реже имеют задолжности по кредитам (почти на 2%). С детьми больше расходов, в том числе и непредвиденных. Поэтому задолжность у таких заемщиков чаще возникает, чем у бездетных.
Количество кредитов для бездетных больше суммы оставшихся категорий кредита. Отделу кредитования нужно больше предлагать кредиты бездетным, т.к. это больше половины рынка и самый низкий процент задолжности по кредитам.

### - Есть ли зависимость между семейным положением и возвратом кредита в срок?

Проделаем тоже самое для семейного положения family_status.

In [394]:
df['credits'] = df['debt'].count() / df['debt'].count()
data_pivot_debt = df.pivot_table(index=['family_status'], columns='credits', values='debt', aggfunc='sum')
data_pivot_credits = df.pivot_table(index=['family_status'], columns='credits', values='debt', aggfunc='count')
data_ratio = data_pivot_debt / data_pivot_credits *100
result = data_pivot_debt.merge(data_pivot_credits, on='family_status')
result = result.merge(data_ratio, on='family_status')
result.set_axis(['Кредиты с задолжностью', 'Количество кредитов', 'Процент задолжности, %'], axis= 'columns', inplace= True)
result.sort_values(by='Процент задолжности, %', ascending= False)

,Кредиты с задолжностью,Количество кредитов,"Процент задолжности, %"
family_status,,,
Не женат / не замужем,274,2810,9.750890
гражданский брак,388,4150,9.349398
женат / замужем,931,12339,7.545182
в разводе,85,1195,7.112971
вдовец / вдова,63,959,6.569343


### Вывод

Не женатые/не замужние и в гражданском браке чаще имеют задолжность(примерно на 2%), чем в браке. Вероятно, в браке оба помогают друг другу, в том числе и с финансовыми проблемами, поэтому задолжность возникает реже.
Количество кредитов в браке больше суммы оставшихся категорий кредита. Отделу кредитования нужно больше предлагать кредиты в браке, т.к. это больше половины рынка и один из самых низких процентов задолжности по кредитам.

### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Проделаем тоже самое для  уровня дохода income_group.

In [395]:
df['credits'] = df['debt'].count() / df['debt'].count()
data_pivot_debt = df.pivot_table(index=['income_group'], columns='credits', values='debt', aggfunc='sum')
data_pivot_credits = df.pivot_table(index=['income_group'], columns='credits', values='debt', aggfunc='count')
data_ratio = data_pivot_debt / data_pivot_credits *100
result = data_pivot_debt.merge(data_pivot_credits, on='income_group')
result = result.merge(data_ratio, on='income_group')
result.set_axis(['Кредиты с задолжностью', 'Количество кредитов', 'Процент задолжности, %'], axis= 'columns', inplace= True)
result.sort_values(by='Процент задолжности, %', ascending= False)

,Кредиты с задолжностью,Количество кредитов,"Процент задолжности, %"
income_group,,,
От 108075.0 до 142594.0 руб.,380,4352,8.731618
От 142594.0 до 196593.0 руб.,552,6374,8.660182
Меньше 108075.0 руб.,432,5421,7.969009
От 196593.0 руб.,377,5306,7.105164


### Вывод

Для заемщиков с доходом меньше 200т.р.(3 верхние строчки) риск задолжности по кредиту самый высокий от 8 до 9%. Можно сказать, что в этой категории люди больше подвержены риску лишиться дохода и не выплачить по кредиту. 
После уровня 200 т.р. задолжность возникает реже примерно в 7 % случаев, что говорит о более устойчивом финансовом состоянии заемщиков  и , вероятно, о диверсификации доходов, этой категориии. 

### - Как разные цели кредита влияют на его возврат в срок?

Проделаем тоже самое для  уровня дохода цели кредита purpose_update.

In [396]:
df['credits'] = df['debt'].count() / df['debt'].count()
data_pivot_debt = df.pivot_table(index=['purpose_update'], columns='credits', values='debt', aggfunc='sum')
data_pivot_credits = df.pivot_table(index=['purpose_update'], columns='credits', values='debt', aggfunc='count')
data_ratio = data_pivot_debt / data_pivot_credits *100
result = data_pivot_debt.merge(data_pivot_credits, on='purpose_update')
result = result.merge(data_ratio, on='purpose_update')
result.set_axis(['Кредиты с задолжностью', 'Количество кредитов', 'Процент задолжности, %'], axis= 'columns', inplace= True)
result.sort_values(by='Процент задолжности, %', ascending= False)

,Кредиты с задолжностью,Количество кредитов,"Процент задолжности, %"
purpose_update,,,
автомобиль,403,4306,9.359034
образование,370,4013,9.220035
свадьба,186,2323,8.006888
недвижимость,782,10811,7.233373


### Вывод

Кредиты на автомобиль и образование чаще имеют задолжность, чем на свадьбу и недвижимость. Вероятно, кредиты на недвижемость и свадьбу более обдуманные решения и чаще принимаюся совместно с кем то, поэтому и процент возврата кредита выше.  
Количество кредитов по недвижимости равно сумме оставшихся категорий кредита. Отделу кредитования нужно больше предлагать кредиты на недвижимость, т.к. это половина рынка и самый низкий процент задолжности по кредитам.

### Проведем дополнительные исследования по возрасту и образованию

### - Как возраст влияет на его возврат кредита в срок?

Категоризируем столбец age по возрасту, применив метод apply(), написав функцию age_influence, принимающую значение возраста и возвращающая значение нового столбца age_group.

In [397]:
def age_influence(age):
    if age <= 30 :
        return 'от 18 до 30 лет'
    elif 31 <= age < 40:
        return 'от 31 до 40 лет'
    elif 41 <= age < 50:
        return 'от 41 до 50 лет'
    elif 51 <= age < 60:
        return 'от 51 до 60 лет'
    else:
        return 'от 60 лет'
    
df['age_group'] = df['dob_years'].apply(age_influence)

Проделаем тоже самое для возрастных категорий age_group.

In [398]:
df['credits'] = df['debt'].count() / df['debt'].count()
data_pivot_debt = df.pivot_table(index=['age_group'], columns='credits', values='debt', aggfunc='sum')
data_pivot_credits = df.pivot_table(index=['age_group'], columns='credits', values='debt', aggfunc='count')
data_ratio = data_pivot_debt / data_pivot_credits *100
result = data_pivot_debt.merge(data_pivot_credits, on='age_group')
result = result.merge(data_ratio, on='age_group')
result.set_axis(['Кредиты с задолжностью', 'Количество кредитов', 'Процент задолжности, %'], axis= 'columns', inplace= True)
result.sort_values(by='Процент задолжности, %', ascending= False)

,Кредиты с задолжностью,Количество кредитов,"Процент задолжности, %"
age_group,,,
от 18 до 30 лет,403,3717,10.842077
от 31 до 40 лет,498,5125,9.717073
от 41 до 50 лет,364,4847,7.509800
от 51 до 60 лет,258,4144,6.225869
от 60 лет,218,3620,6.022099


### Вывод

Заемщики в возрасте от 18 до 40, в 10% случаев имеют задолжность по кредитам. Чем старше возраст заемщика, тем ниже падает процент по задолжности. Вероятно, это связано с более сознательным и ответственным подходом к кредиту, а также большая часть 40+ людей в браке, что влияет на вероятность возникновения задолжности с положительной стороны.
В целом распределение количества кредитов по возрастным рамкам не сильно отличается.

### - Как образование влияет на его возврат кредита в срок?

Проделаем тоже самое для столбца образование education:

In [399]:
df['credits'] = df['debt'].count() / df['debt'].count()
data_pivot_debt = df.pivot_table(index=['education'], columns='credits', values='debt', aggfunc='sum')
data_pivot_credits = df.pivot_table(index=['education'], columns='credits', values='debt', aggfunc='count')
data_ratio = data_pivot_debt / data_pivot_credits *100
result = data_pivot_debt.merge(data_pivot_credits, on='education')
result = result.merge(data_ratio, on='education')
result.set_axis(['Кредиты с задолжностью', 'Количество кредитов', 'Процент задолжности, %'], axis= 'columns', inplace= True)
result.sort_values(by='Процент задолжности, %', ascending= False)

,Кредиты с задолжностью,Количество кредитов,"Процент задолжности, %"
education,,,
начальное,31,282,10.992908
неоконченное высшее,68,744,9.139785
среднее,1364,15171,8.990838
высшее,278,5250,5.295238
ученая степень,0,6,0.000000


### Вывод

Чем более образован заемщик, тем ниже вероятность возникновения задолжности. Почти в 2 раза процент задожности у заемщиков с высшим образованием, чем начальное, неоконченное высшее и среднее. Образованныее люди более сознательны, что касается кредитов.
Но заемщиков с высшим образованием в 3 раза меньше. А в сумме обе эти категории состовляют около 95%.

Далее сведем полученные данные по возрасту и образованию в сводную таблицу и проанализируем данные.  
Используем метод pivot_table() с назанием столбцов по возрасту и значениями по задолжности.

In [400]:
data_total_debt = df.pivot_table(index=['education'], columns='age_group', values='debt', aggfunc='sum')
data_total_credits = df.pivot_table(index=['education'], columns='age_group', values='debt', aggfunc='count')
data_total_ratio = data_total_debt / data_total_credits *100
total = result.merge(data_total_ratio, on='education')
total.fillna(0).sort_values(by='Процент задолжности, %', ascending= False) #избавимся от NaN в ячейке ['от 18 до 30 лет','ученая степень']

,Кредиты с задолжностью,Количество кредитов,"Процент задолжности, %",от 18 до 30 лет,от 31 до 40 лет,от 41 до 50 лет,от 51 до 60 лет,от 60 лет
education,,,,,,,,
начальное,31,282,10.992908,16.666667,20.000000,13.636364,3.703704,4.761905
неоконченное высшее,68,744,9.139785,9.063444,9.500000,13.131313,9.615385,1.612903
среднее,1364,15171,8.990838,13.357900,11.454653,8.050491,6.545790,6.642729
высшее,278,5250,5.295238,6.582278,5.216198,5.096661,4.853129,4.075691
ученая степень,0,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Вывод

Больший интерес предоставляет уровень образования: средний и высшее, т.к. в сумме это 95% заемщиков. Из таблицы видно, что процент задолжности падает по мере увеличения возраста и уровня образования. То есть статистику для среднего образования портят заемщики в возрасте от 18 до 40, далее процент падает на целых 50% к 50-ти годам. Падение у заемщиков с высшим образованием к 50-ти годам незначительное, составляет всего 0,5%.

## Шаг 4. Общий вывод

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**  
Да, есть. Заемщики с детьми чаще имеют задолжность, чем без детей. С детьми больше расходов, в том числе и непредвиденных. Поэтому задолжность у таких заемщиков чаще возникает, чем у бездетных.
- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**  
Да, есть. Не женатые/не замужние и в гражданском браке чаще имеют задолжность, чем в браке. Вероятно, в браке оба помогают друг другу, в том числе и с финансовыми проблемами, поэтому задолжность возникает реже.
- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**  
Да, есть. Наибольшая задолжность около 8-9 % приходит на заемщиков с уровнем дохода до 200 т.р. После уровня 200 т.р. задолжность возникает в 7 % случаев, что говорит о более устойчивом финансовом состоянии заемщиков  и , вероятно, о диверсификации доходов, этой категориии. 
- **Как разные цели кредита влияют на его возврат в срок?**  
Кредиты на автомобиль и образование чаще имеют задолжность, чем на свадьбу и недвижимость. Вероятно, кредиты на недвижимость и свадьбу более обдуманные решения и чаще принимаюся совместно с кем то, поэтому и процент возврата кредита выше.

- **Дополнительно проведены исследования по возрасту и образованию:**  
Чем старше возраст заемщика, тем ниже падает процент по задолжности. Вероятно, это связано с более сознательным и ответственным подходом к кредиту.  
Чем более образован заемщик, тем ниже вероятность возникновения задолжности. Почти в 2 раза процент задожности у заемщиков с высшим образованием, чем начальное, неоконченное высшее и среднее. Образованныее люди более сознательны, что касается кредитов.

**В целом каждый пункт имеет влияние на вероятность возникновения задолжности по кредиту в той или ной мере. По первым 4 пунктам разница составляет порядка 2%, тем неменее необходимо заложить коеэффициент по каждому пункту при одобрении кредита.**

**Так же в ходе исследований был определен портрет идеального заемщика. Это - состоящий в браке, бездетный, с высшим образованием, от 40 лет, цель кредита - недвижимость. По данным критериям самый низкий процент верояность возникновения задолжности.**






### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.